#### Types of Trees

A tree is a data structure composed of nodes:
- Each tree has a **root node** (as far as the programming definition is concerned)
- The root node has zero or more **child nodes**.
- Each child node has zero or more child nodes, and so on
- A node is called a **leaf node** when it has no children

The tree **cannot contain cycles**. The nodes may or may not be in a particular order, they could have any data type as values, and they may or may not have links back to their parent nodes.

A very simple class definition for Node and a wrapping Tree class is shown below. **Note**: in interviews we typically do not use a tree class as it doesn't make things simpler.

In [2]:
from __future__ import annotations
from typing import List

# A binary tree would have left and right nodes
class Node:
    def __init__(self, name: str, children: List[Node]):
        self.name = name
        self.children = children
        
class Tree:
    def __init__(self, root: Node):
        self.root = root

Here are some tidbits to watch out for when dealing with trees:

##### Trees vs. Binary Trees
A binary tree is a tree in which each node has up to two children. Not all trees are binary trees.

<img src="assets/binary-tree.png" width="400">

There are many occasions where you may not want to have a tree that is a binary tree. For instance, for a tree representing phone numbers, you might use a 10-ary tree.

##### Binary Tree vs. Binary Search Tree
A **binary search tree** is a binary tree in which **all left descendents <= n < all right descendents**. This must be tree for each node n, including all of its descendents.
- This definition can vary slightly with respect to equality, under some definitions the BST cannot have duplicate values. In others, the duplicates can be only on the right or on either side.

<img src="assets/binary-search-tree.png" width="300">

An implementation of a BST is shown below.

In [30]:
class TreeNode:
  def __init__(self, val):
    self.val = val
    self.left = None
    self.right = None
  
  def __str__(self):
    return str(self.val)
  
  __repr__ = __str__

class BinarySearchTree:
  def __init__(self):
    self.root = None
  
  def insert(self, val):
    if not self.root:
      self.root = TreeNode(val)
      return
    
    self.recursive_insert(self.root, val)
    
  def recursive_insert(self, node: TreeNode, val):
    if val <= node.val:
      if node.left is None:
        node.left = TreeNode(val)
      else:
        self.recursive_insert(node.left, val)
    else:
      if node.right is None:
        node.right = TreeNode(val)
      else:
        self.recursive_insert(node.right, val)

##### Balanced vs Unbalanced
A balanced tree does not mean that the left and right subtrees are exactly the same (that would be a **perfect binary tree**). In reality, a balanced tree means that it isn't *overly imbalanced*. It's balanced enough to ensure O(log n) time for insert and find, but not necessarily perfectly balanced. Two common types of balanced trees are red-black trees and AVL trees.

<img src="assets/balanced-vs-un-balanced-tree.png" width="400">

##### Complete and Full Binary Tree
A **complete binary tree** is fully filled, except for perhaps the last level. To the extent that the last level is filled, it is filled left to right.<br>
A **full binary tree** is a binary tree were each node has either zero or two children. There is no nodes that only have one child.

<img src="assets/complete-and-full-binary-tree.png" width="400">

##### Perfect Binary Tree
A **perfect binary tree** is one where all interior nodes have two children and all leaf nodes are the same level. A perfect binary has exactly $2^k$-1 nodes where k is the number of levels. **Note**: perfect trees are rare in interviews.

<img src="assets/perfect-binary-tree.png" width="400">

#### Binary Tree Traversal

There are two types of binary tree traversals, **depth first search (DFS)** and **breadth first search (BFS)**, both having time complexities of **O(n)**. 

#### Depth-First Search (DFS)
DFS traverses as far down the tree as possible before backtracking. The space complexity for a binary tree DFS is **O(depth) = O(log n)**, because at any given time there will only be *d* nodes in the stack. There are three methods for DFS, covered below both iteratively and recursively.

##### In-Order Traversal
Visit the left branch, then the current node, and finally the right branch. When performed on a binary search tree, this visits the nodes in ascending order, hence the name.

Iterative approach:
- Empty stack and current node as root
- Iterate to the left until root is none, pushing each non-null node to the stack on the way
- When left is none, print the node and set current node to the right of the current node
- If stack is empty and root is none, we are complete

In [3]:
# Recursive
def rec_in_order_traversal(root: Node):
    if root != None:
        in_order_traversal(root.left)
        print(root)  # The 'visit'
        in_order_traversal(root.right)
        
# Iterative:
def in_order_traversal(root: Node):
    node_stack = []

    while node_stack or root:
        if root:
            node_stack.append(root)
            root = root.left
        elif node_stack:
            root = node_stack.pop()
            print(root.val)
            root = root.right


##### Pre-Order Traversal (Top-down)
Visit the current node before its child nodes. The root is always the first node visited.

Iterative approach:
- Stack initialized with root
- While the stack is not empty, pop the latest node and print it
- Add the right of the node to the stack if it's not null (printed after left)
- Add the left of the node to the stack if it's not null (printed first)


In [12]:
def rec_pre_order_traversal(root: Node):
    if root != None:
        print(root)
        pre_order_traversal(root.left)
        pre_order_traversal(root.right)
        
def pre_order_traversal(self, root: Node):
    if not root:
        return None

    node_stack = [root]

    while node_stack:
        node = node_stack.pop()
        print(node.val)

        if node.right != None:
            node_stack.append(node.right)
        if node.left != None:
            node_stack.append(node.left)

##### Post-Order Traversal (Bottom-up)
Post-order traversal visits the current node after its child nodes. **Note**: deletion of nodes is in post order.

Iterative approach:<br>

Using two stacks:
- Similar to pre-order, except we use a second stack to store the reverse post order traversal
    - The reverse post order traversal order is similar to that of pre-order except the right node is visited first
- Each time you pop() a node, store it in a new stack

**Note**: Strictly speaking, this is not true post order traversal because you don't traverse the entire left and right subtrees before visiting each node.

Using one stack:
- Traverse to the left appending each node to stack until you reach a leaf node (left and right are null) - do not push leaf node to the stack
    - A leaf node can be None as well, in the event that the parent only has a right subtree
- Append the current node to the result
- Check if the node to the right of the top of the stack
    - If so, pop from the stack and append to the result
    - Note, use a while loop here in case we are deep in a right subtree
- Set current node to the right of the node at the top of the stack

Using one stack and pushing each node twice (simpler):
- Push each node to the stack twice
- Check if the popped node is equivalent to the last in the stack
    - If so, this is the first time visiting this node and we must first add the left and right nodes to the stack
    - If not, this node has already been visited and we can add it to the result


In [6]:
def rec_post_order_traversal(n: Node):
    if n != None:
        post_order_traversal(n.left)
        post_order_traversal(n.right)
        print(n)

# Two stacks
def post_order_traversal(root: Node):
    if not root:
        return

    temp_stack = [root]
    post_order_stack = []
        
    while temp_stack:
        root = temp_stack.pop()
        post_order_stack.append(root.val)

        if root.left:
            temp_stack.append(root.left)
        if root.right:
            temp_stack.append(root.right)

    # Print post order stack in reverse order
    while post_order_stack:
        print(post_order_stack.pop())

# Single stack
def post_order_traversal(root: Node):
    node_stack = []

    while node_stack or root:
        # Traverse to bottom of left subtree
        while not is_leaf_node(root):
            node_stack.append(root)
            root = root.left

        # Append leftmost node's value to result if it's not none
        if root:
            print(root.val)

        # Append parent to result if current node is right child
        while node_stack and root == node_stack[-1].right:
            root = node_stack.pop()
            print(root.val)

        # Set current node to right of parent if node is not empty
        root = None if not node_stack else node_stack[-1].right

def is_leaf_node(node: Node):
    if not node:
        return True
    elif node.left is None and node.right is None:
        return True
    return False

# Single stack pushing each node twice
def post_order_traversal(self, root: Node):
    if not root:
        return

    node_stack = [root] * 2

    while node_stack:
        curr = node_stack.pop()
        if node_stack and curr == node_stack[-1]:
            if curr.right:
                node_stack += [curr.right] * 2
            if curr.left:
                node_stack += [curr.left] * 2
        else:
            # This node has already been visited
            print(curr.val)

#### Breadth-First Search (BFS)
BFS also starts at the node but it traverses the tree one layer at a time. This can commonly be implemented with a queue. The space complexity for BFS is **O(n)** because in a perfect binary tree you could have all (n+1)/2 leaf nodes in the queue. See below for a level order traversal using a queue.

In [15]:
from collections import deque
from typing import List

def level_order(root: Node) -> List[List[int]]:
    # The queue stores the node and its corresponding level
    queue = deque([(root, 0)])
    res = []

    while queue and root:
        root, level = queue.popleft()

        # Append to the corresponding level in the result array
        if level > len(res) - 1:
            res.append([])
        res[level].append(root.val)

        if root.left != None:
            queue.append((root.left, level + 1))
        if root.right != None:
            queue.append((root.right, level + 1))

    return res

#### Binary Heaps

A **min-heap** is a *complete* binary tree (that is, totally filled other than the rightmost elements on the last level) where each node is **smaller** than its children. The root is therefore the minimum element of the tree. **Max-heaps** are essentially equivalent but the elements are in descending order rather than ascending order.

<img src="assets/min-heap.png" width="400">

We have two key operations on a min-heap: insert and extract_min.

##### Insert
**O(log n)**, where n is the number of nodes in the heap.
- Start inserting the element at the next available spot at the bottom (looking left to right)
    - This is done to maintain the complete property
- Fix the tree by swapping the new element with its parent, until we find an appropriate spot for that element.
    - We **bubble up** the minimum element

##### Extract Minimum Element
**O(log n)**, where n is the number of nodes in the heap. Note the minimum element is always the root node.
- Remove the root node and swap it with the last element in the heap (the bottom rightmost element)
- **Bubble down** this element by swapping it with one of its children until the min-heap property is restored
    - When bubbling down, swap with the minimum element to maintain the min-heap ordering (doesn't matter if it is on the left or right)

##### Priority Queues
A priority queue is an abstract data type in which each element has a priority queue associated with it. The higher priority elements are served first before the lower priority ones. A heap is commonly used to implement a priority queue to ensure O(log n) insertion and deletion of an element.

##### Heap Implementation
Implementations of min-heap and max-heap as arrays are shown below. Note that when the root node is stored at index 1 instead of 0, the indices for the parent and children of any node become:
- Parent: $i//2$
- Left child: $2*i$
- Right child: $2*i + 1$

A leaf node will also always have an index greater than *n//2* where n is the total number of nodes in the heap. The first element in the array will occasionally be instead used to store the size of the tree.

In [1]:
class EmptyHeapException(Exception):
    pass

class MinHeap:
    def __init__(self):
        self.heap_list = [None]  # Initialize with an arbitrary value to simplify parent/child node indices
        self.size = 0
        
    def add(self, element):
        self.heap_list.append(element)
        self.size += 1
        # Bubble the element from bottom to top
        self.sift_up(self.size)
    
    # Delete the top (minimum) element of the heap
    def pop(self):
        if self.size == 0:
            raise EmptyHeapException("The heap is empty!")
        
        root = self.heap_list[1]
        # Replace root with bottom rightmost element
        self.heap_list[1] = self.heap_list[-1]
        # Remove the last element
        self.heap_list.pop()
        self.size -= 1
        
        # Bubble down the root element
        self.sift_down(1)

        return root
    
    def peek(self):
        if self.size == 0:
            raise EmptyHeapException("The heap is empty!")

        return self.heap_list[1]

    def sift_down(self, index):
        # While the element is not a leaf node and has a smaller child, swap with the smaller child
        while (index <= self.size // 2) and (self.heap_list[index] < self.heap_list[self.get_min_child(index)]):
            swap = self.get_min_child(index)
            self.heap_list[index], self.heap_list[swap] = self.heap_list[swap], self.heap_list[index]
            index = swap  # Set new index to the index of the smaller child

    def sift_up(self, index: int):
        parent = index // 2
        # While the element is not the root, bubble up as necessary
        while index > 1 and (self.heap_list[index] < self.heap_list[parent]):
            self.heap_list[parent], self.heap_list[index] = self.heap_list[index], self.heap_list[parent]
            index = parent
            parent = index // 2
    
    def get_min_child(self, index):
        # If right child doesn't exist, return left by default
        if (2 * index) + 1 >= self.size:
            return 2 * index
        else:
            if self.heap_list[(2 * index) + 1] < self.heap_list[2 * index]:
                return (2 * index) + 1
            else:
                return 2 * index

    def __str__(self):
        return str(self.heap_list[1:])
            
# Test cases
minHeap = MinHeap()
minHeap.add(3)
minHeap.add(1)
minHeap.add(2)
# [1,3,2]
print(minHeap)
# 1
print(minHeap.peek())
# 1
print(minHeap.pop())
# 2
print(minHeap.pop())
# 3
print(minHeap.pop())
minHeap.add(4)
minHeap.add(5)
# [4,5]
print(minHeap)

[1, 3, 2]
1
1
3
2
[4, 5]


In [18]:
class EmptyHeapException(Exception):
    pass

class MaxHeap:
    def __init__(self):
        self.heap_list = [None]  # Initialize with an arbitrary value to simplify parent/child node indices
        self.size = 0
        
    def add(self, element):
        self.heap_list.append(element)
        self.size += 1
        # Bubble the element from bottom to top
        self.sift_up(self.size)
    
    # Delete the top (minimum) element of the heap
    def pop(self):
        if self.size == 0:
            raise EmptyHeapException("The heap is empty!")
        
        root = self.heap_list[1]
        # Replace root with bottom rightmost element
        self.heap_list[1] = self.heap_list[-1]
        # Remove the last element
        self.heap_list.pop()
        self.size -= 1
        
        # Bubble down the root element
        self.sift_down(1)

        return root
    
    def peek(self):
        if self.size == 0:
            raise EmptyHeapException("The heap is empty!")

        return self.heap_list[1]

    def sift_down(self, index):
        # While the element is not a leaf node and has a smaller child, swap with the smaller child
        while (index <= self.size // 2) and (self.heap_list[index] > self.heap_list[self.get_max_child(index)]):
            swap = self.get_max_child(index)
            self.heap_list[index], self.heap_list[swap] = self.heap_list[swap], self.heap_list[index]
            index = swap  # Set new index to the index of the smaller child

    def sift_up(self, index: int):
        parent = index // 2
        # While the element is not the root, bubble up as necessary
        while index > 1 and (self.heap_list[index] > self.heap_list[parent]):
            self.heap_list[parent], self.heap_list[index] = self.heap_list[index], self.heap_list[parent]
            index = parent
            parent = index // 2
    
    def get_max_child(self, index):
        # If right child doesn't exist, return left by default
        if (2 * index) + 1 >= self.size:
            return 2 * index
        else:
            if self.heap_list[(2 * index) + 1] > self.heap_list[2 * index]:
                return (2 * index) + 1
            else:
                return 2 * index

    def __str__(self):
        return str(self.heap_list[1:])

# Test cases
maxHeap = MaxHeap()
maxHeap.add(1)
maxHeap.add(2)
maxHeap.add(3)
# [3,1,2]
print(maxHeap)
# 3
print(maxHeap.peek())
# 3
print(maxHeap.pop())
# 2
print(maxHeap.pop())
# 1
print(maxHeap.pop())
maxHeap.add(4)
maxHeap.add(5)
# [5,4]
print(maxHeap)

[3, 1, 2]
3
3
1
2
[5, 4]


#### Tries
A trie (sometimes called a **prefix tree**) is a variant of an n-ary tree in which characters are stored at each node. Each path down the tree may represent a word. **\* nodes** (null nodes) are often used to indicate complete words. \* nodes might be special type of child (such as TerminatingTrieNode) or simply a boolean flag within a node. A node in a trie can have anywhere from 1 through ALPHABET_SIZE + 1 children (or 0 through ALPHABET_SIZE if a boolean flag is used instead of a \* node). <br>

A common use for tries is to use them for quick prefix lookups in O(k) time where k is the length of the string.

#### Graphs

A tree is actually a type of graph, but not all graphs are trees. A tree is simply a connected graph without cycles.<br>

A **graph** is a collection of nodes (or **vertices**) with edges between (some of) them.
- A graph can be directed (one-way street) or undirected (two-way street)
- The graph might consist of multiple isolated subgraphs
-   If there is a path between every pair of vertices it is called a **connected** graph
- The graph can also have cycles
  - A graph without cycles is called an **acyclic graph**

##### Terminologies

- **Vertex**: The nodes of a graph
- **Edge**: The conencted between two vertices.
- **Path**: The sequence of vertices from one vertex to another (there can be multiple)
- **Path Length**: The number of edges in a path
- **Cycle**: A path where the start and endpoint are the same vertex
- **Negative Weight Cycle**: In a "weighted graph", if the sum of the weights of all edges of a cycle is a negative value, it is a negative weighted cycle
- **Connectivity**: If there exists at least one path between two vertices, these two vertices are connected.
- **Degree of a Vertex**: The term degree applies to **unweighted graphs**. The degree of a vertex is the number of edges connecting the vertex.
- **In-Degree**: Concept in **directed graphs**, the number of directional edges leading into (or incident *to*) the vertex
- **Out-Degree**: Concept in **directed graphs**, the number of directional edges leading out of (or incident *from*) the vertex

<img src="assets/graphs.png" width="400">

##### Adjacency List

An adjacency list is the most common way to represent a graph. Every vertex (or node) stores a list adjacent vertices. In an undirected graph, and edge like (a, b) would be stored twice, once in a's adjacent vertices and once in b's.<br>

A simple graph node definition could look similar to a tree node, however we use a wrapping Graph class because you can't necessarily reach all the nodes from a single node. 

In [1]:
from __future__ import annotations
from typing import List

class Graph:
  def __init__(self, nodes: List[Node]):
    self.nodes = nodes

class Node:
  def __init__(self, name: str, children: List[Node]):
    self.name = name
    self.children = children

##### Adjacency Matrix
An adjacency matrix is an NxN boolean matrix where N is the number of nodes. A true value (or a 1) at matrix[i][j] indicates an edge from node i to node j. In an undirected graph, an adjacency matrix will be symmetric, while it may not necessarily be in a directed graph.

<img src="assets/adjacency-matrix.png" width="400">

Although the same algorithms like BFS and DFS can be applied to adjacency matrices, they will be slightly less efficient since you have to iterate through all the nodes before identifying neighbours.

##### Graph Search
The two most common ways to search a graph are **depth-first search (DFS)** and **breadth-first search (BFS)**.<br>

In depth-first search, we start at the root (or another arbitrary node) and explore each branch completely before going onto the next branch. We go deep before we go wide. DFS is often preferred if we want to visit every node in the graph.<br>

In breadth-first search, we start at the root (or another arbitrary node) and explore each neighbor before going on to any of their children. We go wide before we go deep. BFS is generally used to **find the shortest path (or just any path)** between two nodes.

##### Depth-First Search (DFS)
In DFS, we visit a node *a* and iterate through each of *a*'s neighbours. When visiting node *b*, *a*'s neighbour, we visit all of *b*'s neighbours before going on to *a*'s other neighbours. The key to DFS of a graph is that we must check if the node has been visited, otherwise we risk getting stuck in an infinite loop. Pseudocode for DFS:

```
void search(Node root):
  if (root == null) return
  visit(root)
  root.visited = true
  for each Node n in root.adjacent:
    if n.visited == false:
      search(n)
```

The time complexity for BFS is **O(V + E)** where V is the number of vertices and E is the number of edges in the graph. The space complexity is **O(V)** since in the worst case you may need to hold all vertices in the stack. See [here](https://en.wikipedia.org/wiki/Depth-first_search#Properties) for an explanation.

##### Breadth-First Search (BFS)
In BFS, node *a* visits each of *a*'s neighbours before visiting any of *their* neighbours. The main point with BFS is that it is not recursive, you will need to use a queue. Pseudocode for BFS:

```
void search(Node root):
  Queue queue = new Queue()
  root.marked = true
  queue.enqueue(root)  // Add to the end of queue

  while !queue.isEmpty():
    Node r = queue.dequeue()  // Remove from the front of the queue
    visit(r)
    for each Node n in r.adjacent:
      if n.marked == false:
        n.marked = true
        queue.enqueue(n)
```

The time complexity for BFS is **O(V + E)** where V is the number of vertices and E is the number of edges in the graph. The space complexity is **O(V)** since in the worst case you may need to hold all vertices in the queue, although for larger graphs it may be represented using the *branching factor*. See [here](https://en.wikipedia.org/wiki/Breadth-first_search#Analysis) for an explanation.

##### Bidirectional Search
Bidirectional search is used to find the shortest path between a source and destination node. It operates by essentially running two simultaneous breadth-first searches, one from each node. When their searches collide, we have found a path which is formed by merging the two paths. Note that if the graph is directed, it searches forward from *s* and backwards from *t*.

To understand why this is faster, consider a graph where every node has at most *k* adjacent nodes and the shortest path from node *s* to *t* has length *d*:
- In traditional BFS, we search *k* nodes in the first level of the search, and *k* more for each of those first *k* nodes ($k^2$ nodes on the second level). We would do this d times, so that's O($k^d$) nodes.
- In bidirectional search, the two searches collide after approximately $d/2$ levels (the midpoint of the path). Each serach visits $k^d/2$ nodes, which is approximately $2k^d/2$ or O($k^{d/2}$) nodes total. 

Bidirectional search is therefore faster by a factor of $k^{d/2}$.


#### Interview Questions

**4.1 Route Between Nodes**
- Perform a BFS on the directed graph and stop if the end node is found
- O(V + E) time and O(V) space

In [14]:
from collections import deque
from typing import List

class Node:
  def __init__(self, name: str):
    self.name = name
    self.adj: List[Node] = []
    self.visited = False

# Resets nodes
def unmark_nodes(nodes: List[Node]):
  for node in nodes:
    node.visited = False

def search(s: Node, e: Node):
  q = deque()
  s.visited = True
  q.append(s)

  # While the queue is not empty
  while q:
    n = q.popleft()

    if n == e:
      return True

    for child in n.adj:
      if not child.visited:
        child.visited = True
        q.append(child)

  return False

n1 = Node("S")  # Start
n2 = Node("B")
n3 = Node("C")
n4 = Node("D")
n5 = Node("Z")
n6 = Node("E")  # End

nodes = [n1, n2, n3, n4, n5, n6]
n1.adj = [n2, n3, n4]
n2.adj = [n6]
n3.adj = [n2, n5]
n4.adj = [n5]


print(f"A search between n1 and n5 should return True: {search(n1, n5)}")
unmark_nodes(nodes)
print(f"A search between n3 and n5 should return True: {search(n3, n5)}")
unmark_nodes(nodes)
print(f"A search between n3 and n4 should return False: {search(n3, n4)}")


A search between n1 and n5 should return True: True
A search between n3 and n5 should return True: True
A search between n3 and n4 should return False: False


**4.2 Minimal Tree**
- The root node must be the middle of the tree since the array is sorted in ascending order
- The middle of the left and right subarrays are the roots of the left and right subtrees, and so on
- Recursively this problem is trivial
- O(n) time (each node must be processed) and O(log n) space (at a maximum, only the depth of the tree needs to be stored)

In [16]:
from typing import List

class TreeNode:
  def __init__(self, val: int):
    self.val = val
    self.left: TreeNode = None
    self.right: TreeNode = None

def minimal_tree(arr: List[int]):
  def recursive_bst(start, end):
    if end < start:
      return None

    mid = (start + end) // 2
    node = TreeNode(arr[mid])
    node.left = recursive_bst(start, mid - 1)
    node.right = recursive_bst(mid + 1, end)

    return node

  # Return the root node    
  return recursive_bst(0, len(arr) - 1)

arr = [1, 3, 7, 8, 9, 12]
root = minimal_tree(arr)

# Print response
tree = level_order(root)

for level in tree:
  print(level)

[7]
[1, 9]
[3, 8, 12]


**4.3 List of Depths**
- All solutions will be **O(n)** time and **O(n)** space because of the length of the result, however some will technically use less space
- Can perform a level order (BFS) traversal which uses a queue (an additional O(n))
- The book also uses modified pre-order traversal (passing the level down to each stack call), and a modified BFS which does not require the extra space of the queue (shown below)

In [32]:
import random

class TreeNode:
  def __init__(self, val: int):
    self.val = val
    self.left: TreeNode = None
    self.right: TreeNode = None
  
  def __str__(self):
    return str(self.val)
  
  __repr__ = __str__

# Modified BFS without using a queue
def list_of_depths(root: TreeNode):
  res = []
  current_list = []  # The current linked list

  if root:
    current_list.append(root)
  
  while len(current_list) > 0:
    res.append(current_list)  # Append the current list to the result
    parents = current_list  # The current list now become the parent nodes
    current_list = []

    for parent in parents:
      if parent.left:
        current_list.append(parent.left)
      if parent.right:
        current_list.append(parent.right)
  
  return res

# Test using BST class from above
bst = BinarySearchTree()
for i in range(8):
  bst.insert(random.randint(0, 25))

res = list_of_depths(bst.root)

for level in res:
  print(level)

[13]
[8, 19]
[0, 10, 18]
[17, 19]


**4.4 Check Balanced**
- Post order traversal (DFS) while tracking the depth of each node's subtree
- Return two values, the max depth at each node and whether the subtree was balanced
- **O(n)** time and **O(log n)** space